In [1]:
import requests
from tqdm import tqdm 
import time
import json
import os

In [2]:
headers = {
    'accept': '*/*',
    'accept-language': 'it-IT,it;q=0.9,en-US;q=0.8,en;q=0.7',
    'content-type': 'application/json',
    'origin': 'https://www.fotmob.com',
    'priority': 'u=1, i',
    'referer': 'https://www.fotmob.com/it',
    'sec-ch-ua': '"Google Chrome";v="125", "Chromium";v="125", "Not.A/Brand";v="24"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"Windows"',
    'sec-fetch-dest': 'empty',
    'sec-fetch-mode': 'cors',
    'sec-fetch-site': 'same-origin',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/125.0.0.0 Safari/537.36',
}


In [3]:
def getMatchComments(game: dict, headers: dict, language: str = 'en') -> list:
  home = game['home']['name']
  away = game['away']['name']
  id = game['id']
  params = {
    'ltcUrl': f'data.fotmob.com/webcl/ltc/gsm/{id}_{language}.json.gz',
    'teams': f'["{home}","{away}"]'}

  game_comment_response = requests.get('https://www.fotmob.com/api/ltc', params=params, headers=headers)
  if game_comment_response.status_code != 200:
    raise Exception(f'Errore nella estrazione dei commenti della partita {home} - {away}. Status code: {game_comment_response.status_code}')
  return game_comment_response.json()['events']


def getLeagues(header: str) -> list:
  response = requests.get('https://www.fotmob.com/api/allLeagues',  headers=headers)
  if response.status_code != 200:
    raise Exception(f'Errore nella estrazione delle competizioni. Status code: {response.status_code}')
  leagues = response.json()
  selected_leagues = [x for x in leagues['popular'] if x['id'] not in [77, 50, 44]]
  selected_leagues += [y for x in leagues['countries'] for y in x['leagues'] if y['id'] ==  61]
  return selected_leagues

def getLeagueMatches(id: int, season: str, headers: dict) -> list:

  params = {
      'id': str(id),
      'season': season,
  }

  response = requests.get('https://www.fotmob.com/api/leagues', params=params, headers=headers)
  if response.status_code != 200:
    raise Exception(f'Errore nella estrazione delle partite del campionato {id}, stagione {season}. Status code: {response.status_code}')
  return response.json()['matches']['allMatches']

In [9]:
def writeJson(dati, percorso_file):
    """
    Scrive un dizionario o una lista di dizionari su un file JSON.

    :param dati: Il dizionario o la lista di dizionari da scrivere su file.
    :param percorso_file: Il percorso del file JSON.
    """
    try:
        with open(percorso_file, 'w', encoding='utf-8') as file:
            json.dump(dati, file, ensure_ascii=False, indent=4)
        #print(f"Dati scritti correttamente su {percorso_file}")
    except Exception as e:
        print(f"Errore durante la scrittura su file: {e}")

In [10]:
selected_seasons = ['2019/2020', '2020/2021', '2021/2022', '2022/2023', '2023/2024']
selected_leagues = getLeagues(headers)
dataset_path = 'Dataset\\Fotmob\\'
languages = {'Bundesliga': 'de', 'La Liga': 'es', 'Serie A': 'it', 'Liga Portugal': 'pt', 'Ligue 1': 'fr'}


with tqdm(total=len(selected_seasons), desc="Processando le stagioni", unit="stagione", ncols=80, bar_format="{l_bar}{bar}| {n_fmt}/{total_fmt} [{elapsed}<{remaining}, {rate_fmt}]") as pbar:
    for season in selected_seasons:
        season_path = season.replace('/', '-')
        path_out = os.path.join(dataset_path, season_path)
        start_time = time.time()
        
        for league in selected_leagues:
            start_league = time.time()
            matches = getLeagueMatches(league['id'], season, headers)
            path_out_league = os.path.join(path_out, league['name'])
            print(f"Inizio estrazione {league['name']} stagione {season} ora inizio {time.time()}")
            for match in matches:

                home = match['home']['name']
                away = match['away']['name']
                round = match['round'].replace('/','_')
                comment = getMatchComments(match, headers)
                if len(comment) == 0 and league['name'] in languages.keys():
                    lang = languages[league['name']]
                    comment = getMatchComments(match, headers, lang)

                filename= f'{round}_{home}_{away}.json'
                path_filename = os.path.join(path_out_league, filename)
                writeJson(comment, path_filename)
            print(f"Fine estrazione {league['name']} stagione {season} ora fine {time.time()}")
                    
        iter_time = time.time() - start_time
        pbar.set_postfix_str(f"Iter time: {iter_time:.2f}s")
        pbar.update(1)
        

Processando le stagioni:   0%|                      | 0/4 [00:00<?, ?stagione/s]

Inizio estrazione Premier League stagione 2020/2021 ora inizio 1718460102.9062421
Fine estrazione Premier League stagione 2020/2021 ora fine 1718460184.5173044
Inizio estrazione Champions League stagione 2020/2021 ora inizio 1718460185.0106492
Fine estrazione Champions League stagione 2020/2021 ora fine 1718460209.9814827
Inizio estrazione LaLiga stagione 2020/2021 ora inizio 1718460211.0339003
Fine estrazione LaLiga stagione 2020/2021 ora fine 1718460290.139651
Inizio estrazione Bundesliga stagione 2020/2021 ora inizio 1718460290.4808903
Fine estrazione Bundesliga stagione 2020/2021 ora fine 1718460355.5840938
Inizio estrazione Serie A stagione 2020/2021 ora inizio 1718460356.004968
Fine estrazione Serie A stagione 2020/2021 ora fine 1718460445.0821517
Inizio estrazione Europa League stagione 2020/2021 ora inizio 1718460445.3847487
Fine estrazione Europa League stagione 2020/2021 ora fine 1718460489.7660267
Inizio estrazione Ligue 1 stagione 2020/2021 ora inizio 1718460490.6417675
Fin

Processando le stagioni:  25%|███▎         | 1/4 [08:57<26:51, 537.24s/stagione]

Fine estrazione Liga Portugal stagione 2020/2021 ora fine 1718460639.7773807
Inizio estrazione Premier League stagione 2021/2022 ora inizio 1718460640.1475646
Fine estrazione Premier League stagione 2021/2022 ora fine 1718460719.9955566
Inizio estrazione Champions League stagione 2021/2022 ora inizio 1718460720.3153472
Fine estrazione Champions League stagione 2021/2022 ora fine 1718460750.3524683
Inizio estrazione LaLiga stagione 2021/2022 ora inizio 1718460750.7436042
Fine estrazione LaLiga stagione 2021/2022 ora fine 1718460828.0820196
Inizio estrazione Bundesliga stagione 2021/2022 ora inizio 1718460828.3442788
Fine estrazione Bundesliga stagione 2021/2022 ora fine 1718460913.7835367
Inizio estrazione Serie A stagione 2021/2022 ora inizio 1718460914.5552459
Fine estrazione Serie A stagione 2021/2022 ora fine 1718461014.116764
Inizio estrazione Europa League stagione 2021/2022 ora inizio 1718461014.5508578
Fine estrazione Europa League stagione 2021/2022 ora fine 1718461044.7057972


Processando le stagioni:  50%|██████▌      | 2/4 [18:41<18:50, 565.12s/stagione]

...

Processando le stagioni:  75%|█████████▊   | 3/4 [30:35<10:32, 632.73s/stagione]

Fine estrazione Liga Portugal stagione 2022/2023 ora fine 1718461937.5971298
Inizio estrazione Premier League stagione 2023/2024 ora inizio 1718461938.2470503
Fine estrazione Premier League stagione 2023/2024 ora fine 1718462016.4895942
Inizio estrazione Champions League stagione 2023/2024 ora inizio 1718462017.459036
Fine estrazione Champions League stagione 2023/2024 ora fine 1718462042.3796046
Inizio estrazione LaLiga stagione 2023/2024 ora inizio 1718462042.9094167
Fine estrazione LaLiga stagione 2023/2024 ora fine 1718462109.7991014
Inizio estrazione Bundesliga stagione 2023/2024 ora inizio 1718462110.256496
Fine estrazione Bundesliga stagione 2023/2024 ora fine 1718462219.9949555
Inizio estrazione Serie A stagione 2023/2024 ora inizio 1718462220.5370722
Fine estrazione Serie A stagione 2023/2024 ora fine 1718462342.5697236
Inizio estrazione Europa League stagione 2023/2024 ora inizio 1718462343.39125
Fine estrazione Europa League stagione 2023/2024 ora fine 1718462368.9311368
Ini

Processando le stagioni: 100%|█████████████| 4/4 [41:22<00:00, 620.62s/stagione]

Fine estrazione Liga Portugal stagione 2023/2024 ora fine 1718462585.031394


: 

In [28]:
def crea_directory_per_stagioni(seasons, base_path):
    """
    Crea le directory per ciascuna stagione se non esistono.

    :param seasons: Lista di stagioni.
    :param base_path: Percorso base dove creare le directory.
    """
    for season in seasons:
        # Crea il percorso completo per la stagione
        season_path = os.path.join(base_path, season)
        try:
            # Crea la directory se non esiste
            os.makedirs(season_path, exist_ok=True)
            print(f"Directory creata o già esistente: {season_path}")
        except Exception as e:
            print(f"Errore durante la creazione della directory {season_path}: {e}")

# Esempio di utilizzo
selected_seasons_path = ['2019-2020', '2020-2021', '2021-2022', '2022-2023', '2023-2024']
dataset_path = 'Dataset\\Fotmob\\'
for season in selected_seasons_path:
    for league in selected_leagues:
        name = league['name']
        path = os.path.join(dataset_path, season)
        print(path)
        season_path = os.path.join(path, name)
        try:
            # Crea la directory se non esiste
            os.makedirs(season_path, exist_ok=True)
            print(f"Directory creata o già esistente: {season_path}")
        except Exception as e:
            print(f"Errore durante la creazione della directory {season_path}: {e}")

#crea_directory_per_stagioni(selected_seasons, 'Dataset\\Fotmob\\')

Dataset\Fotmob\2019-2020
Directory creata o già esistente: Dataset\Fotmob\2019-2020\Premier League
Dataset\Fotmob\2019-2020
Directory creata o già esistente: Dataset\Fotmob\2019-2020\Champions League
Dataset\Fotmob\2019-2020
Directory creata o già esistente: Dataset\Fotmob\2019-2020\LaLiga
Dataset\Fotmob\2019-2020
Directory creata o già esistente: Dataset\Fotmob\2019-2020\Bundesliga
Dataset\Fotmob\2019-2020
Directory creata o già esistente: Dataset\Fotmob\2019-2020\Serie A
Dataset\Fotmob\2019-2020
Directory creata o già esistente: Dataset\Fotmob\2019-2020\Europa League
Dataset\Fotmob\2019-2020
Directory creata o già esistente: Dataset\Fotmob\2019-2020\Ligue 1
Dataset\Fotmob\2019-2020
Directory creata o già esistente: Dataset\Fotmob\2019-2020\Liga Portugal
Dataset\Fotmob\2020-2021
Directory creata o già esistente: Dataset\Fotmob\2020-2021\Premier League
Dataset\Fotmob\2020-2021
Directory creata o già esistente: Dataset\Fotmob\2020-2021\Champions League
Dataset\Fotmob\2020-2021
Directory

In [8]:
selected_leagues = getLeagues(headers)
selected_leagues[-1]

{'id': 61,
 'name': 'Liga Portugal',
 'localizedName': 'Liga Portugal',
 'pageUrl': '/leagues/61/overview/liga-portugal'}

In [7]:
game = getLeagueMatches(55,'2022/2023', headers)[0]
comments = getMatchComments(game, headers, 'it')


In [17]:
from googletrans import Translator
translator = Translator(service_urls=['translate.google.it'])

In [20]:
for comment in comments:
    line = comment['text']
    print(line)
    translated = translator.translate(line, src='it', dest='en')
    print(translated.text)

Grazie per aver seguito con noi la diretta di questa partita, arrivederci al prossimo incontro di Serie A!


ProtocolError: Invalid input ConnectionInputs.RECV_PING in state ConnectionState.CLOSED

In [23]:
comments[-1]

{'teamEvent': 'away',
 'includeInHighLight': True,
 'key': '327ce51f-5f75-49cb-92a0-b204d6b0e27d',
 'type': 'comment',
 'isSubstitution': False,
 'text': 'Presso lo Stadio Giuseppe Meazza\xa0è tutto pronto per Milan-Udinese,\xa0anticipo valevole per la 1ª giornata di Serie A.',
 'time': None,
 'clockIconType': 'Empty',
 'onlyText': True,
 'players': []}